<a href="https://colab.research.google.com/github/hoiyurie/tibame-ml0602/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from urllib.request import urlretrieve
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/train.csv"
urlretrieve(url,"train.csv")
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/test.csv"
urlretrieve(url,"test.csv")


('test.csv', <http.client.HTTPMessage at 0x7fc3f954f4d0>)

In [37]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding="utf-8")
test_df = pd.read_csv("test.csv",encoding="utf-8")

In [38]:
# 先把train data 和test data 兩個表格連結起來
data = pd.concat([train_df, test_df],ignore_index = True) # ingnore_index = True 重置合併後的index
data = data.drop(["PassengerId","Survived"], axis = 1)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [39]:
# 查看空值 data.isna（）
# 查看空值總數 data.isna().sum()
# 對Series做篩選[帶入跟你的資料筆數一樣多True/False list]

na = data.isna().sum()
na[na>0].sort_values(ascending= False) # ascending = False 



Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [40]:
# # apply 

# s = pd.Series([1,2,3])
# def func(n):
#   return n * n 
# s.apply(func)


# # lambda = def 
# # 多行 def
# # 少數幾行 lambda

# s= pd.Series([1, 2, 3])
# def func(n):
#   return n * n
# s.apply(lambda n:n*3)


# # 可以直接寫成
# s= pd.Series([1, 2, 3])
# s.apply(lambda n:n*3)


In [41]:
# 先處理一些數據：

#Cabin 取出首字母，可能代表艙位等級

def cabin_head(s):
  if not pd.isna(s):
    return s[0]
  # else:
  #   return s  # 回傳NAN

data["Cabin"] = data["Cabin"].apply(cabin_head)


In [42]:
# Ticket 可以看出同行人數

dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [50]:
# 補缺失值（類別：最常出現）
# ！！！如果有一堆測試資料，不要重新算，直接補S)

# Embarked  登船點--> 補成 最多人使用的登船點
# data["Embarked"]
most = data["Embarked"].value_counts().idxmax # idxmax 最大值的索引
data["Embarked"] = data["Embarked"].fillna(most)
na = data.isna().sum()
na[na > 0].sort_values(ascending = False)


Cabin    1014
Age       263
Fare        1
dtype: int64